In [12]:
from CSVModifier import aggregate_high_freq_to_low, add_factors, clean_outliers
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd


from Util import fetch_stock_data_finnhub_paginated

In [16]:
SYMBOL = 'SPY'
START_DATE = '2025-05-01'
END_DATE = '2025-05-07'
FINHUB_TOKEN = 'cvop3lhr01qihjtq3uvgcvop3lhr01qihjtq3v00'
HIFRREQ = '15'
LOFREQ = '60'

In [17]:
# 请求数据
df_hiFreq = fetch_stock_data_finnhub_paginated(
    symbol= SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=HIFRREQ, # 单位为分钟
    token=FINHUB_TOKEN,
    chunk_days=25,
    verbose=True
)

df_loFreq = fetch_stock_data_finnhub_paginated(
    symbol= SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=LOFREQ, # 单位为分钟
    token=FINHUB_TOKEN,
    chunk_days=25,
    verbose=True
)


Fetching 2025-05-01 to 2025-05-07... Status: ok
Fetching 2025-05-01 to 2025-05-07... Status: ok


In [18]:
# 检查数据
df_loFreq.tail(5)
# df_hiFreq.tail(5)

,timestamp,open,high,low,close,volume
58,2025-05-06 18:00:00,558.7210,561.091,558.584,561.0510,5144519
59,2025-05-06 19:00:00,560.8710,561.411,558.709,558.8110,12195181
60,2025-05-06 20:00:00,558.7810,559.511,556.809,557.8611,3118577
61,2025-05-06 21:00:00,557.8611,558.811,557.239,557.3710,530932
62,2025-05-06 22:00:00,560.6810,563.921,560.529,562.6310,148606


In [19]:
# 把高频转换成低频集合
df_hiFreq_agg = aggregate_high_freq_to_low (df_hiFreq)

# 计算因子
df_loFreq_factored = add_factors (df_loFreq)

# 拼接
df_merged = pd.merge (df_loFreq_factored, df_hiFreq_agg, how='left', on='timestamp')
df_merged = clean_outliers(df_merged, z_thresh=10, show_msg=True)

检测到 0 个异常值在 open 列
检测到 0 个异常值在 high 列
检测到 0 个异常值在 low 列
检测到 0 个异常值在 close 列
检测到 0 个异常值在 volume 列
检测到 0 个异常值在 vwap 列
检测到 0 个异常值在 EMA5_open 列
检测到 0 个异常值在 EMA10_open 列
检测到 0 个异常值在 EMA20_open 列
检测到 0 个异常值在 EMA5_high 列
检测到 0 个异常值在 EMA10_high 列
检测到 0 个异常值在 EMA20_high 列
检测到 0 个异常值在 EMA5_low 列
检测到 0 个异常值在 EMA10_low 列
检测到 0 个异常值在 EMA20_low 列
检测到 0 个异常值在 EMA5_close 列
检测到 0 个异常值在 EMA10_close 列
检测到 0 个异常值在 EMA20_close 列
检测到 0 个异常值在 RSI_open 列
检测到 0 个异常值在 MACD_value_open 列
检测到 0 个异常值在 MACD_signal_open 列
检测到 0 个异常值在 MACD_histogram_open 列
检测到 0 个异常值在 ROC_open 列
检测到 0 个异常值在 RSI_high 列
检测到 0 个异常值在 MACD_value_high 列
检测到 0 个异常值在 MACD_signal_high 列
检测到 0 个异常值在 MACD_histogram_high 列
检测到 0 个异常值在 ROC_high 列
检测到 0 个异常值在 RSI_low 列
检测到 0 个异常值在 MACD_value_low 列
检测到 0 个异常值在 MACD_signal_low 列
检测到 0 个异常值在 MACD_histogram_low 列
检测到 0 个异常值在 ROC_low 列
检测到 0 个异常值在 RSI_close 列
检测到 0 个异常值在 MACD_value_close 列
检测到 0 个异常值在 MACD_signal_close 列
检测到 0 个异常值在 MACD_histogram_close 列
检测到 0 个异常值在 ROC_close 列
检测到 0 个异常值在 Stoch_K 列
检测

In [21]:
# 检查数值
df_merged[df_merged['close_volatility'] == 0]

df_merged.shape

(43, 50)

In [22]:
import re
path = f"{SYMBOL}_{START_DATE}_{END_DATE}_{LOFREQ}m.csv"
# path = re.sub(r'[^\w\-]', '', path)
df_merged.to_csv(path, index=False)